# Science under the Stars Vitessce Vizualization

Directory Structure:
```sh
vitessce_suts/
├── data/
│   ├── data.fcs
│   └── embedded_data.zarr *
└── scripts/
    ├── process_fcs.ipynb
    └── vitessce.ipynb *
```


In [10]:
from pathlib import Path

import numpy as np
from matplotlib.colors import to_rgb
from vitessce import (
    AnnDataWrapper,
    VitessceConfig,
)
from vitessce import Component as cm
from vitessce import CoordinationType as ct

## Create Vitessce Config

In [11]:
cancer_cells = "Cancer Cells"
queen_bee_cells = "Queen Bee Cells"
healthy_cells = "Healthy Cells"

In [12]:
def color_name_to_int(color_name):
    """Converts a color name to a list of integers representing the color in RGB

    Parameters
    ----------
    color_name : str
        The color to convert

    Returns
    -------
    list[int]
        The integer RGB values
    """
    rgb = to_rgb(color_name)
    return (np.array(rgb) * 255).astype(int).tolist()

In [13]:
vit_adata_path = Path("../data/embedded_data.zarr")

In [5]:
vc = VitessceConfig(schema_version="1.0.15", name="Science Under the Stars", description="The Dataset for the event")

dataset = vc.add_dataset(name="Dataset").add_object(
    AnnDataWrapper(
        adata_path=vit_adata_path,
        obs_embedding_paths=["obsm/X_umap_2D"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/cell_type_patient"],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X",
        coordination_values={
            "obsType": "cell",
            "featureType": "gene",
            "featureValueType": "expression",
        },
    )
)

In [7]:
scatterplot_umap = vc.add_view(view_type=cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(view_type=cm.OBS_SETS, dataset=dataset)
markers = vc.add_view(view_type=cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(view_type=cm.HEATMAP, dataset=dataset)
cell_set_sizes = vc.add_view(view_type=cm.OBS_SET_SIZES, dataset=dataset)

vc.link_views_by_dict(
    views=[cell_sets, heatmap, cell_set_sizes],
    input_val={
        ct.OBS_COLOR_ENCODING: "cellSetSelection",
        ct.OBS_SET_SELECTION: [
            ["Cell Type", cancer_cells],
            ["Cell Type", queen_bee_cells],
            ["Cell Type", healthy_cells],
        ],
        ct.OBS_SET_COLOR: [
            {"path": ["Cell Type", healthy_cells], "color": color_name_to_int("xkcd:grey")},
            {"path": ["Cell Type", queen_bee_cells], "color": color_name_to_int("xkcd:green")},
            {"path": ["Cell Type", cancer_cells], "color": color_name_to_int("xkcd:red")},
        ],
    },
    meta=False,
)
vc.layout((scatterplot_umap | (cell_sets / markers)) / (heatmap | cell_set_sizes));

## Open the Web View

In [9]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22Science+Under+the+Stars%22%2C+%22description%22%3A+%22The+Dataset+for+the+event%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22Dataset%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8001%2FA%2F0%2F5a2e8deb-b76a-43bb-98cf-c589458a4f46.adata.zarr%22%2C+%22options%22%3A+%7B%22obsEmbedding%22%3A+%5B%7B%22path%22%3A+%22obsm%2FX_umap_2D%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22UMAP%22%7D%5D%2C+%22obsSets%22%3A+%5B%7B%22name%22%3A+%22Cell+Type%22%2C+%22path%22%3A+%22obs%2Fcell_type_patient%22%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22X%22%7D%2C+%22featureLabels%22%3A+%7B%22path%22%3A+%22sup_names%22%7D%7D%2C+%22coordinationValues%22%3A+%7B%22obsType%22%3A+%22cell%22%2C+%22featureType%22%3A+%22gene%22%2C+%22featureValueType%22%3A+%22expression%22%7D%7D%5D%7D%5D%2C+%22coordinationSp